In [5]:
pip install optuna

  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached PyYAML-6.0-cp38-cp38-win_amd64.whl (155 kB)
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11106 sha256=84872c07afbf8a085c36f6c3d952964f7ab2814002fd189fea886a2343b7470f
  Stored in directory: c:\users\ac\appdata\local\pip\cache\wheels\7f\1a\65\84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Import basic libraries
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# from cuml.ensemble import RandomForestClassifier as cuRFC
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import train_test_split

import optuna

In [7]:
# Load the data set

df = pd.read_csv('UCI_Credit_Card.csv')

In [8]:
df = df.rename(columns={'default.payment.next.month': 'def_pay', 
                        'PAY_0': 'PAY_1'})
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,def_pay
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [9]:
features = ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
X = df[features].copy()


In [11]:
# create the target variable
y = df['def_pay'].copy()

In [ ]:
# split the df into train and test, it is important these two do not communicate during the training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
# this means we will train on 80% of the data and test on the remaining 20%.

In [13]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1200)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 0, 100)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 0, 100)
    max_features = trial.suggest_float('max_features', 0.1, 0.9)
    bootstrap = trial.suggest_categorical("bootstrap", ["True", "False"])
    
    
    clf = cuRFC(n_estimators=n_estimators, criterion=criterion, max_depth = max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf,max_features = max_features, bootstrap = bootstrap)
    
    gbm = clf.fit(X_train, y_train)
    predictions = gbm.predict(X_test) 
    accuracy = accuracy_score(y_test, y_pred = predictions)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Number of finished trials: ', len(study.trials))
print("Best trial: ", study.best_trial.params)

[I 2022-01-20 16:07:52,317] A new study created in memory with name: no-name-93e4fbe9-54e2-4d7c-8f00-7b19b6c7c2ed
[W 2022-01-20 16:07:52,321] Trial 0 failed because of the following error: NameError("name 'cuRFC' is not defined")
Traceback (most recent call last):
  File "c:\users\ac\anaconda3\envs\py38\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\AC\AppData\Local\Temp/ipykernel_16056/1275183642.py", line 11, in objective
    clf = cuRFC(n_estimators=n_estimators, criterion=criterion, max_depth = max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf,max_features = max_features, bootstrap = bootstrap)
NameError: name 'cuRFC' is not defined


NameError: name 'cuRFC' is not defined